# Feature Extraction from Spotify

In this notebook, we are going to get audio features of tracks on MSD genre dataset and merge with these features with those we extracted before. 

In [1]:
import spotipy
import requests
import spotipy.util as util
import pprint, json

Environment variables for getting token is defined as below:

In [2]:
CLIENT_ID = '25e3cc7f42ad41d3b0a394e0658ec33a'
CLIENT_SECRET = '5ab7fd87ffb8417d8ce83cbfde8283ad'
scope = 'user-library-read playlist-modify user-read-private'

## Getting authorization token

In [3]:
def get_token(username, scope, client_id, client_secret):
    return util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri='http://localhost/')

## Searching for track with name and its artist

In [4]:
def get_track_id_by_name_and_artist(token, track_name, artist_name):
	response = requests.get('https://api.spotify.com/v1/search',
    				headers={ 'authorization': "Bearer " + token}, 
    				params={ 'q': 'track:' + track_name +  ' artist:' + artist_name, 'type': 'track' })
	return json.loads(response.text)["tracks"]["items"][0]["id"]

## Getting audio features of tracks

In [5]:
def get_audio_features(sp, track_id_list):
	return sp.audio_features(tracks=track_id_list)

Since we have created functions we need, we are ready to go! Let's get a token.

In [6]:
username = '11131118133'
token = get_token(username, scope, CLIENT_ID, CLIENT_SECRET)
sp = spotipy.Spotify(auth=token)

## Getting other features

In [3]:
import pandas as pd
features = pd.read_csv('features_with_lda.csv', sep="\t")
features.head

<bound method NDFrame.head of        index                 genre            track_id       artist_name  \
0          0  classic pop and rock  TRNJTPB128F427AE9F  Blue Oyster Cult   
1          1  classic pop and rock  TRLFJHA128F427AEEA  Blue Oyster Cult   
2          2  classic pop and rock  TRCQZAG128F427DB97  Blue Oyster Cult   
3          3  classic pop and rock  TRSIZRN128F427DB95  Blue Oyster Cult   
4          4  classic pop and rock  TRDYTEO128F427DB90  Blue Oyster Cult   
5          5  classic pop and rock  TRKSICM128F427DB8B  Blue Oyster Cult   
6          6  classic pop and rock  TRUMVIV128F934656F  Blue Oyster Cult   
7          7  classic pop and rock  TRDEAWL128F933D96A  Blue Oyster Cult   
8          8  classic pop and rock  TRJPXIV128F426697A  Blue Oyster Cult   
9          9  classic pop and rock  TRXWSIN128F4265A40  Blue Oyster Cult   
10        10  classic pop and rock  TRYWZCP128F4266CB7  Blue Oyster Cult   
11        11  classic pop and rock  TRFBLMY128F426BEC9  Bl